# DAT480 Grp 1 pattern matching preprocessing

In [2]:
import numpy as np
from matplotlib import pyplot as plt 
import subprocess
import os
from os import listdir
from os.path import isfile, join
import re
import string
import random
from tqdm import tqdm

## Initalize files

In [157]:
path = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/DAT480/Rules_div_by_length/"
r_hashed = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content_hashed.txt")
r_length = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content_length.txt")
r_text = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content.txt")
t_hashed = r_hashed.read().splitlines(False)
t_hashed = np.array(t_hashed, dtype = str)
t_length = r_length.read().splitlines(False)
t_length = np.array(t_length, dtype=int)
t_text = r_text.read().splitlines(False)
t_text = np.array(t_text, dtype=str)
r_hashed.close()
r_length.close()

rules_hashed = np.array([t_length, t_hashed, t_text]).T
rules_len_sort = rules_hashed[np.argsort(t_length,axis=0)]

## Create generic arrays to use for HLS

In [252]:
arr = (np.array(np.unique(rules_len_sort[:,0],return_counts=True),dtype=int))
indices=np.argsort(arr[0])
#specify the lengths you would like to use, 3,8 will yield the lengths 4,5,6,7,8
lengths_to_use = range(3,8)
sorted_arr = np.array([arr[0][indices], arr[1][indices]])

#write the number of elements per unique length in the order of the lengths
f = open(path+"elements.h","w+")
f.write("const uint16_t elements [] = {\n")
for element in sorted_arr[1][lengths_to_use]:
    f.write(str(element)+",\n")
f.write("};")

#write the unique lengths in order
f = open(path+"lengths.h","w+")
f.write("const uint16_t lengths [] = {\n")
for k in sorted_arr[0][lengths_to_use]:
    f.write(str(k)+",\n")
f.write("};")
f.close()    

f = open(path+"ruleset.h", "w+")
f.write("const uint32_t rules [] = {\n")
for rule in rules_len_sort[(rules_len_sort[:,0].astype(int) >= lengths_to_use[0]+1) & (rules_len_sort[:,0].astype(int) <len(lengths_to_use)+lengths_to_use[0]+1)][:,1]:
    f.write("0x"+str(rule)+",\n")
f.write("};")
f.close()

f = open(path+"str_ruleset.h", "w+")
curr_len = 0
for length in sorted_arr[0][lengths_to_use]:
    f.write("const char str_rules_"+str(length)+ "[]["+str(length)+"] = {\n")
    for el in rules_len_sort[rules_len_sort[:,0] == str(length)]:
        f.write("{")
        for byte in el[2]:
            f.write("'"+str(byte)+"',")
        f.write("},\n")
    f.write("};\n")
f.close()


f = open(path+"switch.h", "w+")
switch_initalize = "#define LONG_SWITCH switch (curr_max_len){"
switch_close = "default: continue;}"
f.write(switch_initalize)
for length in sorted_arr[0][lengths_to_use]:
    case_body = "case "+str(length)+": for(int i = 0; i<"+str(length)+"; i++){if(stream_mem[BUFFER_WIDTH-1][head-i] != str_rules_"+str(length)+"[curr_idx][i]){match = 1;}}break;"
    f.write(case_body)
f.write(switch_close)
f.close()

## Create test vectors for the testbench, i.e generate input reference and golden reference

In [167]:
number_of_rules_to_test = 100
number = 0
lengths = sorted_arr[0]
hash_bin = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Prototyping/CRC/hash.o" #precompiled binary to compute the hashes
input_vec = open(path+"test_vec_input.txt","w+")
gold_vec = open(path+"test_vec_gold.txt","w+")
input_str = ""
#mix randomness with rules from the ruleset
for i in range(1,number_of_rules_to_test*2):
    if(i%3 != 0):
        length = random.randrange(1, 20)
        randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
        input_vec.write(randomstr)
        input_str += randomstr

    else:
        number = random.randrange(start_at,breakif)
        text = rules_len_sort[number][2]
        input_str += text 
        input_vec.write(text)
        
input_vec.close()
for i in tqdm(range(len(input_str))):
    max_length = -1
    curr_element = []
    for length in lengths[3:8]:
        args = (hash_bin,input_str[i:i+length])
        popen = subprocess.Popen(args, stdout=subprocess.PIPE)
        popen.wait()
        output = popen.stdout.read().decode().strip("\n")
        res = np.where(rules_len_sort[(rules_len_sort[:,0].astype(int) >= 4) & (rules_len_sort[:,0].astype(int) <9)] == np.char.lower(output))[0] #efficiency ftw
        if(res.size != 0 and max_length < length):
            max_length = length
            curr_element = res[0]
    if(curr_element):
        gold_vec.write(str(curr_element)+"\n")

gold_vec.close()

100%|██████████| 1761/1761 [02:05<00:00, 14.04it/s]


In [205]:
rules_len_sort[(rules_len_sort[:,0].astype(int) >= 4) & (rules_len_sort[:,0].astype(int) <9)][:,1]

array(['962d5b03', '5fd6e450', '7497f02c', '2590a133', '91661b28',
       'd800c4ea', '1e0ce220', '6da3ff96', '4495a843', '9be2d36b',
       '90d8e0ee', 'fc02037b', '3d3b7ef1', 'dcf54b45', 'f06fbfb6',
       'f26bd46c', 'b68adff9', '221b99db', 'c1c3c8e8', 'd4ffd652',
       'd9abdacc', '891c8d4b', 'd235316b', '72daccca', 'b46f482b',
       '5094f40c', '3cdfce37', 'd8d22452', '24b49a7d', '101c8d05',
       'f8e53990', '697a25c', '6c1f8139', '190260a4', 'd89e4f4d',
       '2d4feae4', 'a296361f', '61cd9a2e', '84112de6', 'a25fc5e4',
       '210da9b4', 'c91b1f15', '70fe12ee', 'daec87ea', '7b2f5636',
       'b11a76ba', 'a36e12fc', '5b37df5d', 'a2332e4c', 'c16221dd',
       '3b6769ae', 'ff3b89b0', 'ce4b5ded', 'bd7c8b61', 'bb281f0d',
       '635a9daf', 'cb81cc64', 'be1feff9', '8eaf7b', 'c879ffd2',
       '25d53dfd', '2158414f', 'c65feb48', '1919ea4b', '90054cce',
       '17cdacfb', 'bfa718d7', '6601f115', 'a15faa1f', '8bffad55',
       'cf2713fd', 'bb063bfd', '7cabfd0f', '9be3e0a3', 'ffd2a23e'